<a href="https://colab.research.google.com/github/GleiconMaior/GleiconMaior/blob/main/PCA_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import pickle
import seaborn as sns

# Inicializando o PCA e o NearestNeighbors fora do loop para evitar recriação
pca = PCA(n_components=3)
neigh = NearestNeighbors(radius=0.50)

def process_point(point, df):
    idxs_neigh = neigh.kneighbors([point], return_distance=False)
    df_ = df.iloc[np.squeeze(idxs_neigh)]

    pca.fit(df_.values)

    rugosidade_1 = pca.singular_values_[2]/pca.singular_values_[0]*pca.singular_values_[1]
    rugosidade_2 = pca.singular_values_[2]/pca.singular_values_[0]
    rugosidade_3 = pca.singular_values_[2] / max(pca.singular_values_[0], pca.singular_values_[1])

    n1 = np.array([0, 0, 1]) # normal ẑ
    n2 = pca.components_[2] # normal à superfície das duas 1as componentes principais

    theta = np.arccos(n1.dot(n2) / (np.linalg.norm(n1) * np.linalg.norm(n2)))

    return rugosidade_1, rugosidade_2, rugosidade_3, theta

    # Carregamento dos dados
flname = 'nomedoarquivo'
df = pd.read_csv(flname, sep=' ', header=None, names=['x', 'y', 'z'], skiprows=1)
df

df_red = df.sample(frac=0.5)
plt.scatter(df['x'], df['y'], c=df['z'], s=0.5)
plt.scatter(df_red['x'], df_red['y'], c='k', s=0.5)

# Aplicando o NearestNeighbors aos dados completos
neigh.fit(df.values)

# Processamento paralelo
results = Parallel(n_jobs=-1)(delayed(process_point)(point, df) for point in df_red.values)

# Extrair os resultados em listas separadas
list_rugosidades_1, list_rugosidades_2, list_rugosidades_3, list_theta = zip(*results)

def scale_values(values):
  """
  Função para escalar os valores para a escala de 1 a 10.

  Args:
      values: Uma lista de valores a serem escalados.

  Returns:
      Uma lista de valores escalados.
  """
  min_value = min(values)
  max_value = max(values)
  return [(value - min_value) / (max_value - min_value) * 10 for value in values]

  # Escalar os valores para a escala de 1 a 10
list_rugosidades_1_scaled = scale_values(list_rugosidades_1)
list_rugosidades_2_scaled = scale_values(list_rugosidades_2)
list_rugosidades_3_scaled = scale_values(list_rugosidades_3)
list_theta_scaled = scale_values(list_theta)

# Função de plotagem reutilizável
def plot_data(x, y, c, title, xlabel, ylabel, cbar_label):
    plt.scatter(x, y, c=c, s=30)
    colorbar = plt.colorbar()
    colorbar.set_label(cbar_label, labelpad=10)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

# Plotando os dados escalados
plot_data(df_red['x'], df_red['y'], list_rugosidades_1_scaled, 'Análise de Rugosidade - Escala 1 a 10', 'X (m)', 'Y (m)', 'Índice de Rugosidade 1')
plot_data(df_red['x'], df_red['y'], list_rugosidades_2_scaled, 'Análise de Rugosidade - Escala 1 a 10', 'X (m)', 'Y (m)', 'Índice de Rugosidade 2')
plot_data(df_red['x'], df_red['y'], list_rugosidades_3_scaled, 'Análise de Rugosidade - Escala 1 a 10', 'X (m)', 'Y (m)', 'Índice de Rugosidade 3')
plot_data(df_red['x'], df_red['y'], list_theta, 'Ângulo dos pontos com relação ao eixo XY - Escala 1 a 10', 'X (m)', 'Y (m)', 'Ângulo (radianos)')

# Calculando a média da rugosidade no local
mean_rugosidade = np.mean(list_rugosidades_1)
mean_rugosidade_2 = np.mean(list_rugosidades_1_scaled)
# Imprimindo o valor médio da rugosidade
print("Valor médio da rugosidade no local (real, normalizada):", mean_rugosidade, mean_rugosidade_2)

# Variância das rugosidades não escalonadas
variance_rugosidade_1 = np.var(list_rugosidades_1)
variance_rugosidade_2 = np.var(list_rugosidades_2)
variance_rugosidade_3 = np.var(list_rugosidades_3)
variance_theta = np.var(list_theta)

# Variância das rugosidades escalonadas
variance_rugosidade_1_scaled = np.var(list_rugosidades_1_scaled)
variance_rugosidade_2_scaled = np.var(list_rugosidades_2_scaled)
variance_rugosidade_3_scaled = np.var(list_rugosidades_3_scaled)
variance_theta_scaled = np.var(list_theta_scaled)

# Imprimir as variâncias
print("Variância da rugosidade 1 (real, normalizada):", variance_rugosidade_1, variance_rugosidade_1_scaled)
print("Variância da rugosidade 2 (real, normalizada):", variance_rugosidade_2, variance_rugosidade_2_scaled)
print("Variância da rugosidade 3 (real, normalizada):", variance_rugosidade_3, variance_rugosidade_3_scaled)
print("Variância do ângulo (real, normalizada):", variance_theta, variance_theta_scaled)

# Adicionar informações de média e variância às imagens
def plot_data_with_info(x, y, c, title, xlabel, ylabel, cbar_label, mean_roughness, var_roughness):
    plt.scatter(x, y, c=c, s=30)
    colorbar = plt.colorbar()
    colorbar.set_label(cbar_label, labelpad=10)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if mean_roughness is not None:
      plt.text(0.05, 0.9, f'Média da Rugosidade: {mean_roughness:.4f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
    else:
      plt.text(0.05, 0.9, 'Média da Rugosidade: Not applicable', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
    plt.text(0.05, 0.8, f'Variância da Rugosidade: {var_roughness:.4f}', transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
    plt.show()

# Plotando os dados escalados com as informações adicionais
plot_data_with_info(df_red['x'], df_red['y'], list_rugosidades_1_scaled, 'Análise de Rugosidade - Escala 1 a 10', 'X (m)', 'Y (m)', 'Índice de Rugosidade 1', mean_rugosidade, variance_rugosidade_1_scaled)
#plot_data_with_info(df_red['x'], df_red['y'], list_rugosidades_2_scaled, 'Análise de Rugosidade - Escala 1 a 10', 'X (m)', 'Y (m)', 'Índice de Rugosidade 2', mean_rugosidade_2, variance_rugosidade_2_scaled)
#plot_data_with_info(df_red['x'], df_red['y'], list_rugosidades_3_scaled, 'Análise de Rugosidade - Escala 1 a 10', 'X (m)', 'Y (m)', 'Índice de Rugosidade 3', mean_rugosidade_3, variance_rugosidade_3_scaled)
#plot_data_with_info(df_red['x'], df_red['y'], list_theta, 'Ângulo dos pontos com relação ao eixo XY - Escala 1 a 10', 'X (m)', 'Y (m)', 'Ângulo (radianos)', variance_theta)


fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Heatmap code using axes[0, 0] (or directly in sns.heatmap)
sns.heatmap(
    data=np.array(list_rugosidades_2_scaled).reshape(-1, 1),
    ax=axes[0, 0],  # Reference the subplot using axes
    cmap='viridis',
    vmin=0,
    vmax=10,
)


# Criando a figura
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Histograma para rugosidade 1
sns.histplot(data=list_rugosidades_1_scaled, ax=axes[0, 0])
axes[0, 0].set_title('Índice de Rugosidade 1')

# Histograma para rugosidade 2
sns.histplot(data=list_rugosidades_2_scaled, ax=axes[0, 1])
axes[0, 1].set_title('Índice de Rugosidade 2')

# Histograma para rugosidade 3
sns.histplot(data=list_rugosidades_1_scaled, ax=axes[1, 0])
axes[1, 0].set_title('Índice de Rugosidade 3')

# Histograma para ângulo
sns.histplot(data=list_theta, ax=axes[1, 1])
axes[1, 1].set_title('Ângulo dos pontos com relação ao eixo XY')

# Ajustando a figura
fig.tight_layout()
plt.show()

faixas_rugosidade = {
    "<0.01": 2,
    "0.01-0.025": 3,
    "0.025-0.035": 4,
    "0.035-0.05": 5,
    ">0.05": 6,
}

def obter_resistencia_rolando(mean_rugosidade):
    for faixa, resistencia in faixas_rugosidade.items():
        if faixa.startswith("<"):
            if mean_rugosidade < float(faixa[1:]):
                return resistencia
        elif faixa.endswith(">"):
            if mean_rugosidade > float(faixa[:-1]):
                return resistencia
        else:
            min_valor, max_valor = [float(valor) for valor in faixa.split("-")]
            if min_valor <= mean_rugosidade < max_valor:
                return resistencia
    raise ValueError(f"Mean_rugosidade ({mean_rugosidade}) fora das faixas definidas.")

# Exemplo de uso
resistencia = obter_resistencia_rolando(mean_rugosidade)
print(f"Mean rugosidade: {mean_rugosidade:.4f} - Resistência ao rolamento: {resistencia}")